### Import & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
from pathlib import Path
project_dir = Path().resolve().parent
sys.path.append(str(project_dir))

In [3]:
from pipeline import generate_signals, prepare_feature_data, train_and_predict, ensure_db_initialized
from data.db_utils import get_connection

from config import signal_config, target, tscv

### Load data

In [5]:
DB_PATH = '../data/crypto_data.db'
conn = get_connection(DB_PATH)

start_date = '2025-08-01'

In [6]:
price_query = f"""
SELECT * 
from kline 
where datetime >= '{start_date}' and symbol = '{target}'
"""

pred_query = f"""
select symbol, datetime, actuals, predicted 
from signals
where datetime >= '{start_date}' and symbol = '{target}'
"""

In [9]:
price_df = pd.read_sql_query(
        price_query,
        conn,
        parse_dates=['datetime']
    ).sort_values('datetime').set_index(['symbol', 'datetime'])

pred_df = pd.read_sql_query(
        pred_query,
        conn,
        parse_dates=['datetime']
    ).sort_values('datetime').set_index('datetime')

### Fit predict

In [6]:
from datetime import datetime, timedelta
START_DELTA = timedelta(days=45)
model_start_date = (datetime.utcnow() - START_DELTA).strftime('%Y-%m-%d %H:%M:%S')

In [7]:
DB_PATH = '../data/crypto_data.db'
conn = get_connection(DB_PATH)

In [8]:
df_processed, price_all = prepare_feature_data(conn=conn, target=target, model_start_date=model_start_date)

In [9]:
pred_df = train_and_predict(df_processed, cv = tscv)

### Generate signals

In [11]:
signals_df = generate_signals(pred_df=pred_df, price_all=price_all, target=target)

In [12]:
signals_df.tail(5)

,symbol,datetime,actuals,predicted,zscore,raw_signal,vol_filter,filtered_signal,position,signal_reversal,final_signal,model_name,strategy_name
91,ETHUSDT,2025-08-13 20:00:00,0.000759,-0.000445,-1.378957,-1,True,-1,-1,0,0,rf-reg,zscore_atr_v1
92,ETHUSDT,2025-08-13 21:00:00,0.002785,0.000840,0.897803,0,True,0,-1,0,0,rf-reg,zscore_atr_v1
93,ETHUSDT,2025-08-13 22:00:00,0.003166,0.001055,1.178963,1,True,1,1,1,1,rf-reg,zscore_atr_v1
94,ETHUSDT,2025-08-13 23:00:00,-0.006913,0.001415,1.659074,1,True,1,1,0,0,rf-reg,zscore_atr_v1
95,ETHUSDT,2025-08-14 00:00:00,0.004298,0.000200,-0.326457,0,True,0,1,0,0,rf-reg,zscore_atr_v1
